# Projet: comment choisir le meilleur modèle d'apprentissage automatique pour un problème donné?

Déroulement 
- réalisation en binôme 
- A rendre : fichier jupyter + éventullement rapport d'analyse
- Date de livraison : 30 Mars 2022

Dans ce projet nous allons traiter différents problèmes nécessitant l'utilisation des méthodes d'apprentissage automatique.
Le but est de choisir pour un jeu de données, représentant un problème donné, le bon modèle d'apprentissage automatique compatible. Cette tâche englobe notamment le choix du meilleur pré-traitement à appliquer aux jeux de données ainsi que les bons paramètres du modèle d'apprentissage automatique.

Ainsi, Résoudre le problème par le clustering revient à fournir une représentation optimale par des clusters du jeu de données associé. En classification supervisée, résoudre le problème signifie produire le modèle prédictif le plus efficace. Nous vous informons qu'un problème dont le jeu de données ne contient pas de labels est résolu globalement par les méthodes de clustering. Si les labels sont présents, dans ce cas, la classification supervisée est souhaitable. Dans un but pédagogique, nous utilisons le même ensemble labélisé de données pour la classification (supervisé) et clustering (ML non supervisé)

Pour mener cette tâche, certaines étapes sont nécessaires.

- La première étape donne un apperçu sur les jeux de données auxquelles nous nous intéressons. 
- Les étapes de 2 à 4 demandent à développer des fonctions nécessaires à la préparation des jeux de données. 
- A partir de l'étape 5, on procède à la recherche du modèle optimal d'apprentissage automatique pour chacun des problèmes.

Dans ce projet nous allons traiter différents problèmes nécessitant l'utilisation des méthodes d'apprentissage automatique.
Le but est de choisir pour un jeu de données, représentant un problème donné, le bon modèle d'apprentissage automatique compatible. Cette tâche englobe notamment le choix du meilleur pré-traitement à appliquer aux jeux de données ainsi que les bons paramètres du modèle d'apprentissage automatique.

Ainsi, Résoudre le problème par le clustering revient à fournir une représentation optimale par des clusters du jeu de données associé. En classification supervisée, résoudre le problème signifie produire le modèle prédictif le plus efficace. Nous vous informons qu'un problème dont le jeu de données ne contient pas de labels est résolu globalement par les méthodes de clustering. Si les labels sont présents, dans ce cas, la classification supervisée est souhaitable. Dans un but pédagogique, nous utilisons le même ensemble labélisé de données pour la classification (supervisé) et clustering (ML non supervisé)

Pour mener cette tâche, certaines étapes sont nécessaires.

- La première étape donne un apperçu sur les jeux de données auxquelles nous nous intéressons. 
- Les étapes de 2 à 4 demandent à développer des fonctions nécessaires à la préparation des jeux de données. 
- A partir de l'étape 5, on procède à la recherche du modèle optimal d'apprentissage automatique pour chacun des problèmes.

##  <font color = "darkgreen">1- Description des jeux de données</font>

Dans ce projet, nous travaillerons sur des problèmes réels que nous cherchons à résoudre au moyen de l'apprentissage automatique :

* GCM: diagnostic de cancer multiclasses (prostate, pancréans...) , chaque obervation est une représentation d'un cancer via 16063 signatures d'expression de gènes tumoraux. Le cancer pourrait avoir plusieurs representations et donc plusieurs observations.

* Diabetes: diagnostic de diabètes. Une obervation représente les caractéristiques d'un patient (âge, pression, poids...) atteinte ou non du diabète.

* Wall-robot-navigation: mouvements du robot SCITOS G5 naviguant dans une pièce en suivant le mur dans le sens des aiguilles d'une montre, pendant 4 tours, mesurés à l'aide de 24 capteurs à ultrasons disposés en cercle autour de sa "taille". Chaque obervation représente un mouvement: Avancer,  Léger virage à droite, Virage serré à droite ou Tournez légèrement à gauche (le code peut être fourni )

* Japenese-vowel: cet ensemble de données enregistre 640 séries chronologiques de 12 coefficients de cepstre LPC provenant de neuf locuteurs masculins (https://fr.wikipedia.org/wiki/Cepstre). Chaque obervation représente un son d'une voyelle exprimé par un locuteur donné.

## <font color = "darkgreen">2- Chargement des jeux de données</font>

Nous découpons chaque jeu de données en X et y. Pour ce faire, 

Développez une fonction qui charge le jeu de données depuis un fichier  et retourne deux dataframes (un pour les caractéristiques et l'autre pour la classe/labels)

IL vous revient de bien analyser la structure pour determiner quelles sont les caractériqtiques et quels les calsses correspondantes (y). Néaumoins, voici quelques précisions :

#### Notes : 

* Tous les formats sont de type csv
* La colonne relative à la classe/label est généralement à position 0 ou à la dernière position
* La colonne classe n'a pas le même nom dans les différents jeux de données.
* La colonne classe n'est forcément numérique, dans ce cas la conversion s'avère nécessaire. 

In [95]:
import re
import pandas as pd
import numpy as np
import sklearn as skn

def read_csv_parse_labels(path: str, isFirstColumn=False):
  df = pd.read_csv(path, low_memory=False)
  y = None
  if isFirstColumn:
    y = df.iloc[:, 0]
  else:
    y = df.iloc[:, -1]

  labels = y.unique()
  mapping = dict(zip(labels, range(len(labels))))
  y = y.map(mapping)
  df.pop(y.name)
  return df, y, labels

## <font color = "darkgreen">3- Traitement des valeurs nulles/vides</font>


* Analysez tous les jeux de données, vérifiez si dans certaines observations il existe des valeurs nulles/vides
* Renvoyez le nombre d'obervations qui y sont concernées

In [96]:
'''
We normalize the data by replacing all values that matches "nan" or "null" case insensitive by Numpy.nan.
Then we convert all the columns to be floats.
'''
def normalizeData(df: pd.DataFrame) -> pd.DataFrame:
  return pd.DataFrame(df).replace('(?i).[Nan|Null].', value=np.nan, regex=True).apply(lambda x: x.astype(float))

def assert_is_null_or_nan(df: pd.DataFrame):
  nulls, _ = np.where(pd.isnull(df))
  return len(nulls)

 Ces observations nécessitent l'une des opérations suivantes:
* 1- remplacer les valeurs nulles/vides par la moyenne des valeures des observations pour la caratéristique concernée, à répéter pour chaque  caratéristique.
* 2- même opération mais cette-fois ci en calculant le maximum
* 3- même opération mais cette-fois ci en calculant le minimum
* 4- supprimer les observations contenant les valeurs nulles/vides.

 Développez une fonction (qu'on appelera processNan) qui prend en paramètres le jeu de données (dataframe) et un entier, qu'on appelera typeOp, compris dans [1,4]
 En fonction de la valeur de typeOp, vous appliquez l'une des quatre opérations sur le jeu de données fourni en paramètre.

In [97]:
def processNan(df: pd.DataFrame, typeOp: 1 | 2 | 3 | 4):
  temp = pd.DataFrame(df)
  result = {
    1: lambda x: x.fillna(x.mean()),
    2: lambda x: x.fillna(x.max()),
    3: lambda x: x.fillna(x.min()),
    4: lambda x: x.dropna(),
  }[typeOp](temp)

  return result

## <font color = "darkgreen">4- Traitement des caractéristiques</font>


 Avant de faire appel à un modèle d'apprentissage automatique sur un jeu de données, l'étude de corrélation entre caractéristiques pourrait être menée. Si un couple de caractéristiques ayant une très forte corrélation alors l'une des deux caractéristiques est supprimée.

D'après vous, pourquoi on effectue cette suppression?

In [98]:
'''
De nombreux algorithmes dépendent des outils de l’algèbre linéaire et de la résolution des systèmes linéaires.
Les caractéristiques corrélées créent une matrice et des singularités sous-déterminées dans les algorithmes qui les utilisent
(GLM et leurs cousins pénalisés).
Les méthodes de descente en gradient seront difficiles, car ces fonctionnalités sont des lignes parallèles/hyperplans.
Utiliser d’abord des méthodes de réduction de la dimensionnalité peut aider à réduire ce problème,
car elles trouvent une représentation dimensionnelle inférieure des données (et réduisent la multicollinéarité dans la même base).
'''

'\nDe nombreux algorithmes dépendent des outils de l’algèbre linéaire et de la résolution des systèmes linéaires.\nLes caractéristiques corrélées créent une matrice et des singularités sous-déterminées dans les algorithmes qui les utilisent\n(GLM et leurs cousins pénalisés).\nLes méthodes de descente en gradient seront difficiles, car ces fonctionnalités sont des lignes parallèles/hyperplans.\nUtiliser d’abord des méthodes de réduction de la dimensionnalité peut aider à réduire ce problème,\ncar elles trouvent une représentation dimensionnelle inférieure des données (et réduisent la multicollinéarité dans la même base).\n'

 L'un des moyens pour mesurer la corrélation entre deux caractéristiques est l'utilisation de la mesure de Spearman. cette dernière mesure la force et la direction de la relation n liant les deux caractéristiques.  En ce qui concerne la force de la relation, la valeur du coefficient de corrélation renvoyé par Spearman varie entre +1 et -1.  Une valeur de ± 1 indique un degré de relation parfait entre les deux variables.  Plus la valeur du coefficient de corrélation se rapproche de 0, plus la relation entre les deux variables sera faible.  La direction de la relation est indiquée par le signe du coefficient ; un signe + indique une relation positive et un signe - une relation négative. Si le coefficient est équivant à +1 (repectivement -1) alors il signifie que plus les valeurs d'une caractéristique est grande plus les valeurs de l'autre caractéristique est grande (respectivement petite). 

 Développez une fonction qui retourne, pour une caractéristique donnée, les indices de toutes les caractéristiques dont le coefficient de corrélation retourné par la mesure de Spearman dépasse un certain seuil qu'on appelera t (entré en paramètre). 


#### Notes: 
Vous pourriez utiliser la mesure de Spearman proposée par Scipy(https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html). Attention, nous nous intéressons seulement au coefficient de corrélation retournée dans la variable corrélation.  





In [99]:
from scipy import stats


def spearmanGetIndexes(df: pd.DataFrame, t: float):
  result = stats.spearmanr(df).correlation
  temp = np.array(result)

  # On remplace la diagonale toujours égale a 1 par -1 pour la condition en dessous.
  for i in range(len(temp)):
    temp[i][i] = -1
  
  result = np.where(temp > t)
  return result[0]


 Développez une fonction qui supprime d'un jeu de données des caractéristiques à partir de leur indice renseignés en paramètres sous forme de liste. Le but ici est de supprimer les caractéistiques non nécessaires.
 
 
 On peut, supprimer si on le souhaite, les caractériqtiques à partir des noms de colonnes au lieu des indices de colonnes.

In [100]:
def removeIndices(df: pd.DataFrame, indices):
  temp = pd.DataFrame(df)
  temp = temp.drop(df.columns[indices], axis=1)
  return temp


Développez une fonction appelée processCor qui :
- parcourt les caractéristiques 
- et supprime à chaque fois les caractéristiques qui ont 
un coefficient de corrélation avec la caractéristique courante dépassant le seuil t. 

Utilisez pour cela les deux fonctions developpées précedement dans la partie 4.

In [101]:
def processCor(df: pd.DataFrame, t: float):
  return removeIndices(df, spearmanGetIndexes(df, t))

## <font color = "darkgreen">5- Clustering </font>


 Dans cette partie, nous allons associer à un problème (jeu de données) le meilleur modèle de clustering. En conséquence, nous considérons que les jeux de données n'ont pas de labels ou de colonne classe, autrement dit on s'intéresse seulement aux caractéristiques (features). Dans ce cadre, on se focalisera que sur les méthodes de clustering :
- Dbscan(https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN)
- Clustering agglomératif(https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering)
- K-means
#### Notes:
* Un modèle est différent d'un algorithme(méthode). Dans le contexte d'apprentissage automatique,  un algorithme est une procédure exécutée sur un jeu de données. Le modèle est le résultat produit par l'agorithme. 
* Pour fournir un résultat (modèle), l'algorithme pourrait se baser sur un ensemble d'hyperparamètres qui lui sont fournis en entrée. Un hyperparamètre est une variable qui prend une valeur parmi un ensemble ou une infinité de valeurs, son but est de contrôler le processus d'apprentissage automatique (par exemple, le clustering ou la classification supervisée). 
* Différentes valeurs des hyperparamètres pourraient aboutir à différents modèles

 Créez une collection de jeux de données appelée original_datasets. La clé correspond au nom du jeu de données et la valeur correspondante est un tuple (X,y) où X est l'ensemble des caractéristiques et y l'ensemble des labels.  
#### Note:
Utilisez la fonction développée dans la partie 2.

In [111]:
files = ["data/diabetes.csv", "data/GCM.csv", "data/JapaneseVowels.csv", "data/wall-robot-navigation.csv"]
original_datasets = []

for file in files:
  X, y, labels = read_csv_parse_labels(file)
  original_datasets.append((X, y, file))

Nous allons préparer les jeux de données avant leur consommation par les méthodes de clustering. Tout d'abord créez une copie de  original_datasets qu'on appelera datasets. Ensuite traitez chaque jeu de données de datasets avec les fonctions processNan et processCor. On fixera typeOp à 1 et t à 0.7.

In [112]:
datasets = np.array(original_datasets, dtype=object)

for i in range(len(datasets)):
  dataset = datasets[i][0]
  dataset = normalizeData(dataset)
  dataset = processNan(dataset, 1)
  dataset = processCor(dataset, 0.7)
  datasets[i][0] = dataset

En utilisant la collection datasets, appelez les trois méthodes de clustering sur chaque jeu de données de la collection datasets et évaluez le résultat de  partitionnement(clustering) via:
* Calinski Harabaz(https://scikit-learn.org/stable/modules/generated/sklearn.metrics.calinski_harabasz_score.html#sklearn.metrics.calinski_harabasz_score)
* Davies Bouldin(https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html#sklearn.metrics.davies_bouldin_sc)

#### Notes:
* Sachant qu'on considère que les labels ne sont pas fournis, nous évaluons les résultats aussi par des métriques d'évaluation non supervisées, c'est à dire que les métriques n'ont pas accès aux labels pour proposer un score de performance.
* On considère que meilleur est le score des métriques d'évaluation mieux le problème associé au jeu de données est résolu.
* Lors de l'appel des méthodes de clustering, dans cette partie on modifiera pas les valeurs des hyperparamètres, on se contentera des valeurs fixés par défaut par scikit-learn.

Associez à chaque paire "jeu de données et méthode de clustering" les scores retournés par les métriques d'évaluation (calinski, davies).

In [140]:
from sklearn import cluster, metrics

k = 8

kmeansResults = []
dbscanResults = []
agglomerativeClusturingResults = []

results = {
  'kmeans': {},
  'dbscan': {},
  'agglomerativeClustering': {}
}

def evaluate(X, y):
  return (metrics.calinski_harabasz_score(X, y), metrics.davies_bouldin_score(X, y))

for d in datasets:  
  kmeans = cluster.k_means(d[0], k, init='random')[1]
  results['kmeans'][d[2]] = {}
  results['kmeans'][d[2]]['y'] = kmeans
  cal, davies = evaluate(d[0], kmeans)
  results['kmeans'][d[2]]['calinski'] = cal
  results['kmeans'][d[2]]['davies'] = davies

  dbscan = cluster.dbscan(d[0])[1]
  results['dbscan'][d[2]] = {}
  results['dbscan'][d[2]]['y'] = dbscan
  cal, davies = evaluate(d[0], dbscan)
  results['dbscan'][d[2]]['calinski'] = cal
  results['dbscan'][d[2]]['davies'] = davies

  agglomerativeClusturing = cluster.AgglomerativeClustering(k)
  agglomerativeClusturing = agglomerativeClusturing.fit(d[0])
  results['agglomerativeClustering'][d[2]] = {}
  results['agglomerativeClustering'][d[2]]['y'] = agglomerativeClusturing.labels_
  cal, davies = evaluate(d[0], agglomerativeClusturing.labels_)
  results['agglomerativeClustering'][d[2]]['calinski'] = cal
  results['agglomerativeClustering'][d[2]]['davies'] = davies

for k in results:
  print("Results for " + k)
  print("------------------------------------------------------")
  for j in results[k]:
    print(j + " --> " + "(calinski: " + str(results[k][j]['calinski']) + "," + " davies: " + str(results[k][j]['davies']) + ")")
  print("------------------------------------------------------")


[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

Pour chaque jeu de données, quelle est la méthode qui a fourni le meilleur modèle selon les scores calculés dans la question précedente.

## <font color = "darkgreen">6- Peut-on améliorer encore les résultats de clustering? </font>

Dans cette partie nous nous intéressons toujours au clustering mais das une perspective d'amélioration des modèles de clustering produits précedement. L'une des pistes d'amélioration des modèles est de varier les valeurs des hyper-paramètres des méthodes de clustering ainsi que les types de pré-traitement appliqués aux jeux de données. 

Tout d'abord développez une fonction (qu'on appellera getBestModel) qui pour une méthode de clustering donnée et un jeu de données renvoie le meilleur modèle. La fonction prend en paramètre le jeu de données X, le nom de la méthode methodName et une collection d'hyper-paramètres params. Cette collection a pour clé le nom de l'hyper-paramètre et pour valeur la liste des valeurs prises par l'hyper-paramètre. Dans cette fonction, toutes les combinaisons des valeurs des listes dans params seront évaluées et à chaque combinaison on obtiendra un modèle associée au jeu de données et à la méthode de clustering renseignés en paramètre. Chaque modèle est évalué par une métrique d'évaluation au choix (calinski ou davies). 

La collection params comporte les valeurs possibles des hyper-paramètres de la méthode clustering concernée ainsi que les valeurs possibles de typeOp et t. Vous avez le choix de fixer la liste de valeurs de typeOp et t. Vous pourriez commencer par évaluer [1,2,3,4] pour typeOp et [0.5, 0.7, 0.9] pour t. Les valeurs des hyper-paramètres dépendent de la méthode de clustering ciblée:

* Dbscan a plusieurs hyper-paramètres dont les plus influents sont eps, min_samples et metric. Notons que eps est un réel et ne peut dépasser la distance inter-points maximale qui existe dans X, et min_samples est un entier naturel non nul inférieur au nombre d'observations dans X. Metric a par défaut la valeur euclidean, d'autres valeurs sont possibles et consultables sur https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html#sklearn.metrics.pairwise_distances. Vous avez le choix de fixer les listes de ces trois hyper-paramètres. On vous suggère de ne inclure des valeurs dépassant $\sqrt{|X|}$ pour min_samples et la moyenne des distances inter-points pour eps.
* Clustering agglomératif a pour hyper-paramètres les plus sensibles n_clusters, distance_threshold, affinity(équivalent de metric dans Dbscan) et linkage. A savoir que soit n_clusters ou distance_threshold est utilisé pour extraire les clusters finaux, il n'est pas nécessaire d'utiliser les deux. Vous pourriez limiter la valeur de n_clusters à $\frac{\sqrt{|X|}}{2}$ ou limiter la valeur de distance_threshold à la moyenne des distances inter-points.
* K-means a deux hyper-paramètres sensibles: n_clusters(nombre de clusters) et l'initialisation des centroides init{‘k-means++’, ‘random’}. 

#### Note:
Vous aurez à instancier plusieurs fois la collection datasets en fonction des valeurs prises par typeOp et t.



En utilisant la fonction getBestModel, identifiez le meilleur modèle associé à chaque jeu de données et méthode de clustering.

Par rapport à la partie 5, y a t-il eu des améliorations? Justifiez.

## <font color = "darkgreen">7- La classification supervisée</font>


Dans cette partie nous chercherons à résoudre les problèmes par la classification supervisée. Dans ce cas, on considère que les labels (y) sont à notre disposition. Nous nous intéressons à trois méthodes de cette catégorie:
* arbre de décision (https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
* KNN : https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

Instancier la collection datasets dans les mêmes conditions que celles spécifiées dans la partie 5.

Sur chaque jeu de données de la collection datasets appliquez les deux méthodes de classification. A chaque application, évaluez le résultat (modèle) via les métriques d'évaluation suivantes:
* accuracy score (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score)
* balanced accuracy score (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html#sklearn.metrics.balanced_accuracy_score)

#### Notes.
* A l'appel des méthodes de classification, les valeurs des hyper-paramètres doivent restés fixés tel qu'elles sont par scikit-learn. 
* Avant d'appliquer les trois méthodes de classification sur le jeu de données, celui-ci doit être divisé en deux sous-ensembles: ensemble d'entraînement ($~80$% du jeu de données) et l'ensemble de test ($~20$%).

Quelle est la méthode ayant a fourni le meilleur modèle (c'est à dire le meilleur taux de prédiction) pour chaque jeu de données?

## <font color = "darkgreen">8- Peut-on encore améliorer les résultats de classification supervisée</font>


Dans la même perspective que dans la partie 6, nous chercherons ici à améliorer les résultats de la classification en faisant varier les valeurs des hyper-paramètres et les types de pré-traitement des données. 

Reprenez la méthode gestBestModel en faisant en copie de celle-ci et en la nommant getBestModelC. Cette dernière a la même fonction que gestBestModel sauf qu'elle est adaptée pour la classification supervisée: 
* la métrique d'évaluation doit correspondre à l'une des deux métriques de la partie 7. 
* Pour faire face aux problèmes de désiquilibre de répartition de classes de points dans les sous-ensembles d'entraînement et de test, on adoptera la technique d'échantillonage suivante. A partir de chaque jeu de données, on génére 5 sous-ensembles de données disjoints. A chaque itération, un sous-ensemble est utilisé pour le test et les autres pour la phase d'entraînement. Un sous-ensemble doit être utilisé une seule fois dans la phase de test. 

Les listes de valeurs de typeOp et t peuvent être fixées identiquement à celles de la partie 6. 



En utilisant la fonction getBestModelC, identifiez le meilleur modèle associé à chaque jeu de données et méthode de classification?

Par rapport à la partie 7, y a t-il eu des améliorations? Justifiez.